In [1]:
# Kaggle_Data_Extraction
# This class containes the code for
#    1. Reading the flight schedule data from external csv files
#    2. Cleaning of the data read from exteral source
#    3. Exporting the cleaned data into a csv file
#  Detailed description availble through inline comments.

import pandas as pd
import numpy as np
import matplotlib as plt
import json
import re
from re import sub
from pandas.io.json import json_normalize
pd.options.mode.chained_assignment = None



# Flight schedule data read from csv file 'Flight_Schedule.csv' and being stored under dataframe 'inital_flight_schedule_df'
inital_flight_schedule_df = pd.read_csv('Flight_Schedule.csv')

In [2]:
# All Cities data read from csv file 'usa_cities.csv' and being stored under dataframe 'all_cities_df'
all_cities_df = pd.read_csv('usa_cities.csv')
# Creating 'City_ID' column from 'city' column by converting the string to lower case and removing space
all_cities_df['City_ID'] = all_cities_df['city'].str.lower()
all_cities_df['City_ID'] = all_cities_df['City_ID'].str.replace(' ', '')
# Creating dictionary 'all_city_code_dict' by using columns 'city' and 'City_ID'
all_city_code_dict = dict(zip(all_cities_df.city, all_cities_df.City_ID))


# Selected Cities (used in project) data read from csv file 'City.csv' and being stored under dataframe 'cities_df'
cities_df = pd.read_csv('City.csv')
# 'cities_list' created from 'cities_df' by picking only 'city_name' column
cities_list = cities_df['city_name']
# 'city_name_code_dict' created from 'cities_df' by picking 'city_name' and 'city_id' columns
city_name_code_dict= dict(zip(cities_df.city_name, cities_df.city_id))


# Airport codes data read from excel file 'Airport_Codes_With_Cities.xlsx' and being stored under dataframe 'airport_codes_df'
airport_codes_df = pd.read_excel('Airport_Codes_With_Cities.xlsx')
# Creating 'City_ID' column from 'city' column by converting the string to lower case and removing space
airport_codes_df['City_ID'] = airport_codes_df['City'].str.lower()
airport_codes_df['City_ID'] = airport_codes_df['City_ID'].str.replace(' ', '')
# Creating 'airport_code_dict' by making use of 'code' and 'City' columsn from airport_codes_df
airport_code_dict= dict(zip(airport_codes_df.Code, airport_codes_df.City))
# Creating 'airport_city_dict' by making use of 'code' and 'City_ID' columsn from airport_codes_df
airport_city_dict = dict(zip(airport_codes_df.Code, airport_codes_df.City_ID))



# Creating a dataframe from 'airport_codes_df', where only the cities present in 'cities_list'are alon considered
expected_airport_codes_df = airport_codes_df.loc[airport_codes_df['City'].isin(cities_list)]
# Creating a list 'expected_airport_codes_list' by making use of 'Code' column from expected_airport_codes_df
expected_airport_codes_list = expected_airport_codes_df['Code']
# Creating dataframe 'expected_flight_schedule_df' which contains only those Destination airports, whcih are mentioned in 'expected_airport_codes_list'
expected_flight_schedule_df = inital_flight_schedule_df.loc[inital_flight_schedule_df['Dest'].isin(expected_airport_codes_list)]



# creating a new dataframe by copying entire contents from modified_expected_flight_schedule_df 
modified_expected_flight_schedule_df = expected_flight_schedule_df[:]
# Picking up only the necessay columns in 'modified_expected_flight_schedule_df'
modified_expected_flight_schedule_df = expected_flight_schedule_df[['Month', 'DayofMonth', 'CRSDepTime', 'CRSArrTime',
                                                                   'UniqueCarrier','FlightNum','TailNum','AirTime',
                                                                   'Origin','Origin_City_ID','Dest','Destination_City_ID','Distance']]



# Creating a new column 'Origin_City_ID' from 'Origin' column by maing use 'airport_city_dict' with Origin values as input 
modified_expected_flight_schedule_df['Origin_City_ID'] = modified_expected_flight_schedule_df['Origin'].apply(lambda x: airport_city_dict.get(x))
# Creating a new column 'Destination_City_ID' from 'Dest' column by maing use 'airport_city_dict' with Destination values as input 
modified_expected_flight_schedule_df['Destination_City_ID'] = modified_expected_flight_schedule_df['Dest'].apply(lambda x: airport_city_dict.get(x))
#Creating a new column 'Flight_ID' by combining 'FlighNum' and 'TailNum'columns separated by adeliited '_'
modified_expected_flight_schedule_df['Flight_ID']=modified_expected_flight_schedule_df['FlightNum'].astype(str)+'_'+modified_expected_flight_schedule_df['TailNum'].astype(str)



# Modifying existing column 'CRSArrTime' by checking if length of the column value is less than 2 and multiplying it by 10, if true
modified_expected_flight_schedule_df['CRSArrTime'] = modified_expected_flight_schedule_df['CRSArrTime'].apply(lambda x: x*10 if len(str(x)) < 2 else x)
# Modifying existing column 'CRSDepTime' by checking if length of the column value is less than 2 and multiplying it by 10, if true
modified_expected_flight_schedule_df['CRSDepTime'] = modified_expected_flight_schedule_df['CRSDepTime'].apply(lambda x: x*10 if len(str(x)) < 2 else x)
# Changing the datatype of the 'CRSDepTime' column as 'String'
modified_expected_flight_schedule_df['CRSDepTime']= modified_expected_flight_schedule_df['CRSDepTime'].astype(str)
# Changing the datatype of the 'CRSArrTime' column as 'String'
modified_expected_flight_schedule_df['CRSArrTime']= modified_expected_flight_schedule_df['CRSArrTime'].astype(str)



# Creating a new column 'Departure_Time' from 'CRSDepTime' column by converting its corresponding values in time format (hh:mm)
modified_expected_flight_schedule_df['Departure_Time'] = pd.to_datetime(modified_expected_flight_schedule_df['CRSDepTime'], errors='coerce', format='%H%M')
# Creating a new column 'Arrival_Time' from 'CRSArrTime' column by converting its corresponding values in time format (hh:mm)
modified_expected_flight_schedule_df['Arrival_Time'] = pd.to_datetime(modified_expected_flight_schedule_df['CRSArrTime'], errors='coerce', format='%H%M')



#Getting rid of 'date' value from the timestamp associated with 'Departure_Time' and 'Arrival_Time' columns
modified_expected_flight_schedule_df['Departure_Time'] = modified_expected_flight_schedule_df['Departure_Time'].dt.time
modified_expected_flight_schedule_df['Arrival_Time'] = modified_expected_flight_schedule_df['Arrival_Time'].dt.time



# Dropping the unncessary columns from modified_expected_flight_schedule_df
modified_expected_flight_schedule_df.drop(['CRSDepTime', 'CRSArrTime', 
                'FlightNum', 'TailNum', 'Origin', 'Dest'], axis=1, inplace=True)



# Renaming colmnns from 'modified_expected_flight_schedule_df'
modified_expected_flight_schedule_df = modified_expected_flight_schedule_df.rename(columns={'Month': 'Month_Of_The_Year', 'DayofMonth': 'Day_Of_The_Month',
                                                                   'UniqueCarrier': 'Airline_ID','AirTime': 'Air_Time'})


# Data corrections done as per audit validation
# Creating a new dataframe 'final_flight_schedule_df' by removing recrods which have the value as 'AQ' for Airline_ID, as the 
# carrier is defunct
final_flight_schedule_df = modified_expected_flight_schedule_df[modified_expected_flight_schedule_df.Airline_ID != 'AQ']
# Continental Airlines Changed to United Airlines, as the former was mergeed into the latter
final_flight_schedule_df['Airline_ID'] = final_flight_schedule_df['Airline_ID'].apply(lambda x: 'UA' if x=='CO' else x)
# Northwest and Atantic coast airlines changed to delta airlines, as the former was mergeed into the latter
final_flight_schedule_df['Airline_ID'] = final_flight_schedule_df['Airline_ID'].apply(lambda x: 'DL' if x=='DH' else x)
final_flight_schedule_df['Airline_ID'] = final_flight_schedule_df['Airline_ID'].apply(lambda x: 'DL' if x=='NW' else x)
# Atlantic southeastr airlines changed to express jet, as the former was mergeed into the latter
final_flight_schedule_df['Airline_ID'] = final_flight_schedule_df['Airline_ID'].apply(lambda x: 'XE' if x=='EV' else x)
#  AirTran airways changed to southwest airlines, as the former was mergeed into the latter
final_flight_schedule_df['Airline_ID'] = final_flight_schedule_df['Airline_ID'].apply(lambda x: 'WN' if x=='FL' else x)
# American west airlines changed to US airways, as the former was mergeed into the latter
final_flight_schedule_df['Airline_ID'] = final_flight_schedule_df['Airline_ID'].apply(lambda x: 'US' if x=='HP' else x)
# Mesa Airlines changed to American Eagle, as the former was mergeed into the latter
final_flight_schedule_df['Airline_ID'] = final_flight_schedule_df['Airline_ID'].apply(lambda x: 'MQ' if x=='YV' else x)



# Exporting the data from  final_flight_schedule_df dataframe into a csv by name 'Project_Extracted_Flight_Schedule.csv'
final_flight_schedule_df.to_csv("Project_Extracted_Flight_Schedule.csv", index=False)



""""
CONCLUSIONS:

This code reads data taken from an external source 'Kaggle', performs data cleaning, data reformat, stores it in a dataframe and 
finally exports it to a CSV file


CONTRIBUTIONS:

RAJENDRA KUMAR RAJKUMAR - 90%
PRUDHVI CHANDRA SEKHARAMAHANTI - 10%


CITATIONS:

1. https://www.geeksforgeeks.org
2. https://github.com/nikbearbrown/INFO_6210
3. stackoverflow
4. Tutorialspoint

In built functions and methods required for data cleaning, formatting options were referred from above mentioned sites

Percentage of code written - 90%
Percentage of code referred from above mentioned scources - 10%

LICENSE:

Copyright <2019> <RAJENDRA KUMAR RAJKUMAR, PRUDHVI CHANDRA SEKHARAMAHANTI>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


"""